<img src="./img/CCIT_Logo.png" alt="CCit Logo" width=400>

# 06 - Countermeasures against power side-channel analysis 🛡️

## ⚠️ Prerequisites

Hol' up - before continuing, ensure you have done the following:

* ☑ Read the introduction to Side-Channel Analysis in the [02-Intro to SCA](./02-Intro%20to%20SCA.ipynb) notebook.
* ☑ Learn how Simple Power Analysis works in [03-Simple Power Analysis](./03-Simple%20Power%20Analysis.ipynb) notebook.
* ☑ Learn how Differential Power Analysis works in [04-Differential Power Analysis](./04-Differential%20Power%20Analysis.ipynb) notebook.
* ☑ Learn how Correlation Power Analysis works in [05-Correlation Power Analysis](./05-Correlation%20Power%20Analysis.ipynb) notebook.

### 📑 Summary

*In this lab, you'll get an overview of the most common software and hardware conuntermeasures against SCA.*

### 💬 Learning outcomes
* What is the rationale behind the development of countermeasures against SCA
* What are some of the most common *software*, *hardware* and *algorithmic* solutions

## 📑 Table of Contents
0. [Why is there a need for countermeasures?](#why)
1. [What is the rationale to follow?](#rationale)
2. [Transistor-level solutions](#transistor)
3. [Software-level solutions](#sw)
4. [Algorithmic-level solutions](#algorithmic)
5. [Protocol-level solutions](#protocol)
6. [Additional resources](#resources)
6. [Conclusions](#conclusions)

# 0) Why is there a need for countermeasures? <a class="anchor" id="why"></a>
If you read ALL the previous tutorials, you should now understand that SCA, especially in the form of DPA and CPA, is basically "unstoppable". So a question may arise: why should I care about countermeasures against SCA if SCA will always succeed?

What you should also have retained is that SCA is **ideally** unstoppable, but not **in reality**!

> Eve cannot spend hours and hours collecting, aligning and finally computing a massive amount of traces: her time and resources are limited! What is the point in spending, say, 10,000 dollars worth of equipment if the data retrieved from the hardware device (assuming the attack succeeds) can only cover a tenth of that cost? Where's the incentive in completing the attack in such case?

This is what we should aim to by developing countermeasures! We should aim making the attack as complex as possible, at least enough to force the attacker in spending more money and time than she can gain if the attack were successful.

Of course, this is **a generic rule of thumb** that applies in most cases, but **may not apply in ALL cases.** Think about cyberwarfare and State-sponsored attacks: in all these cases, more often than not, the gained information may be considered "invaluable", and the attacker may pursue their objectives no matter the defense mechanism you implemented.

# 1) What is the generic rationale behind most countermeasures? <a class="anchor" id="rationale"></a>
Consider a generic [*signal-to-noise ratio* formula](https://en.wikipedia.org/wiki/Signal-to-noise_ratio):

$$ SNR = \frac{P_{signal}}{P_{noise}} $$

We can use this formula to find similarities with side-channel analysis. In SCA, the goal for an attacker is to extrapolate as much "clean" information (leakage) so as to retrieve useful information from it. As we saw with DPA and CPA, most of the times, the useful part of leakage is limited to small temporal portions, often scrambled up by other device-related information. Not every part of a power trace is useful: some samples may show major uncorrelated contributes coming from the ALU, for instance, or a hardware accelerator embedded in the same die.

Now, consider the term $P_{signal}$ in the above formula: this is our leakage signal. The $P_{noise}$ term, on the other hand, is composed by all the uncorrelated data, not useful for our purposes, but still emitted by the device under attack.

> The goal for an attacker is to maximize his $SNR$, by retrieving as much leakage as he can, isolating the noise from it.

So, having this in mind, what is the goal of a defenser? Look at the above formula, the goal is to *reduce* the $SNR$:
1. Minimize the $P_{signal}$, i.e. minimize the leakage produced by the device;
2. Maximize the $P_{noise}$, i.e. maximize the noise produced by the device, so as to make the leakage difficult to isolate for the attacker.

# 2) Transitor-level solutions <a class="anchor" id="transistor"></a>
This is one of the lowest possible levels where countermeasures can be introduced.  Some of them may focus on reducing the leakage emitted as much as possible. Many others focus on inserting additional noise in the power/electromagnetic traces an attacker can capture.

Such countermeasures require fine-tuning the cryptographic circuits, possibly inserting additional circuitry able to compute useless/garbage operations in parallel to the real crypto operation, so to require additional power consumption translatable in a (hopefully) sufficient level of obfuscation. 

In all cases, the additional protection comes at a cost: additional hardware, which translates into additional power consumption and die area. Moreover, solutions at the transistor level require high levels of expertise and development time: such solutions are expensive and are most suitable to big companies or military-grade necessities.

An example:
> Adding more bus lines than necessary: on a 8-bit microcontroller, add another 8 (useless) bus lines and move 16 bits at a time instead of 8. The additional lines will be designed to operate randomly or in opposition to the original bus lines : in any case their usage will require additional energy to operate, increasing the overall power consumption and obfuscating the "real" power contribute of the "useful" original 8 bus lines.

# 3) Software-level solutions <a class="anchor" id="sw"></a>
Many (if not all) software solutions concentrate on creating additional noise aiming to "obfuscate" the leakage and/or distrupting the temporal reference when multiple traces are collected.

Similarly to the effect produced by misaligned traces (see previous notebook), software countermeasures aim to randomly increase or decrese the time required to complete specific operations, by leveraging useless computations randomly added to the original code flow. Imagine the following scenario:

* Trace \#0: The first round of `AES-128` originally happens from sample \#2000 to sample \#4000, the countermeasure did not inserted any additional delay
* Trace \#1: The same round this time can be recognized from sample \#2000 to sample \#5000, the countermeasure added a totality of 1000 samples during the computations, with respect to the original case 1.
* Trace \#2: The same round this time happened from sample \#2000 to sample \#6423, the countermeasure added a totality of 2423 samples during the computations, with respect to the original case 1.

Now imagine collecting 100 traces that are all messed up like in the above example: how can DPA and CPA retrieve the secret key if there's no more a temporal reference among them?

> Well, SCA and key retrieval is still possible, even in this case. However, the attacker must now be able to recognize the issue and circumvent it, building a (possibly automatic) tool able to recognize all these jitter additions, removing them from the original traces. Therefore, a preprocessing step is required to "clean up" the original traces prior the real DPA/CPA attack

An extremely simple solution to add randomic jitter may be the following:
```c
...
AddRoundKey(State, Key);
//Stop them fancy timing attacks
int wait = rand() % 100000;
for(volatile int i = 0; i < wait; i++){
    ;
}
SubBytes(State, SBox);
...
```

# 4) Algorithmic-level solutions <a class="anchor" id="algorithmic"></a>
These solutions depend on the basic operations required by the algorithm. In the case of `AES`, for instance, modifications may be applied to how the `SubBytes` step operated.

Taking the `SubBytes` operation as an example: recently academia focuses on finding [Substitution Boxes](https://en.wikipedia.org/wiki/S-box) having a different values in the matrix/look-up-table [with respect to `AES` original one](https://en.wikipedia.org/wiki/Rijndael_S-box). Researchers found that changing the integer values of the matrix may lead to a reduced leakage emisison, therefore requiring the attacker to spend additional time collecting and processing a higher number of traces than originally necessary.

# 5) Protocol-level solutions <a class="anchor" id="protocol"></a>
The most effective and least difficult way to address side-channel attacks is to design cryptographic protocols to survive leakage. Since leakage will always be present, even if low, and SCA will always be there exploiting it, why not periodically change the key so that, even if retrieved by the attacker, the damage is reduced as much as possible? In the best case, it may even be possible to change the key before the known leakage emissions become enough for an attacker to retrieve the key.

With such solution, we may have a device that changes the master key after 10 encryptions (hence 10 possible power traces collected by the attacker). So, even if the attacker was able to capture these 10 traces finally retrieving the corresponding key, the key retrieved will be now useless, as the encryption key used by the device starting from encryption \#11 will be a different one! The old key is basically made useless by the automatic key change.

# 6) Additional resources <a class="anchor" id="resources"></a>
1. [Overview] [Paper] [Note on side-channel attacks and their
countermeasures](https://keccak.team/files/NoteSideChannelAttacks.pdf) by the KECCAK Team (Guido Bertoni, Joan Daemen, Micha ̈el Peeters and Gilles Van Assche)
2. [Overview] [Paper] [See section "Preventing DPA"] [Introduction to Differential Power Analysis](https://link.springer.com/content/pdf/10.1007%2Fs13389-011-0006-y.pdf) by Paul Kocher, Joshua Jaffe, Benjamin Jun, Pankaj Rohatgi
3. [Hardware-level solutions] [Paper] [Secure AES Engine with A Local Switched-Capacitor Current Equalizer](https://ieeexplore.ieee.org/document/4977309) by Carlos Tokunaga and David Blaauw
4. [Algorithmic-level solutions] [Paper] [Comparing Sboxes of Ciphers from the Perspective of Side-Channel Attacks](https://eprint.iacr.org/2016/993.pdf) by Liran Lerman, Olivier Markowitch and Nikita Veshchikov

# 7) Conclusions <a class="anchor" id="conclusions"></a>
**This was the final tutorial!** Now you can definitely test your knowledge with the Jeopardy CTFs I prepared for you!

*I hope the explanations were clear and that you learned something new!*

As always, if you have any doubts, suggestions or want to chat a bit, here's where you can find me:
* 🐦 Twitter: `[at]mrcuve0`
* 💬 Discord: `Mrcuve0#4179`
* 💻 GitHub:  `Mrcuve0`
* 📬 Email: `mrcuve0 [at] posteo [d.ot] net`
* 💼 LinkedIn:
> Please, send me a message via the previous methods before asking for my LinkedIn

## See you next time!

---
Copyright ©️ [CINI - Cybersecurity National Lab](https://cybersecnatlab.it/) - Torino, 2022.

> This material is a derivative work of [NewAE's official Jupyter Notebooks](https://github.com/newaetech/chipwhisperer-jupyter), distributed under the open-source GPL license. You can distribute and modify this material (even for commercial trainings), provided you maintain references to this repository and the original authors, and also offer your derived material under the same conditions.


ChipWhisperer is a trademark of NewAE Technology Inc., claimed in all jurisdictions, and registered in at least the United States of America, European Union, and Peoples Republic of China.

All other product names, logos, and brands are property of their respective owners.

The software is provided 'as is', without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.